In [83]:
# getting dependencies
import cohere
import cohere.classify as co_classify
import dotenv
import os
import numpy as np
import pandas as pd 

In [56]:
# happiness vector
day_decoding = ['happy', 'sad']

In [57]:
# setting up client from env file
dotenv.load_dotenv()
co_client = cohere.Client(f'{os.getenv("COHERE_KEY")}')

In [74]:
speech_text = "My laptop missing in class today"

response = co_client.classify(
    inputs = [f"{speech_text}"],
    examples = [
        co_classify.Example('My friend made me smile today', 'happy'),
        co_classify.Example('My family took me out today', 'happy'),
        co_classify.Example('Today my cats gave me a lick', 'happy'), 
        co_classify.Example('I went to the park, but it was rainy and stormy', 'sad'),
        co_classify.Example('At school, I won an award', 'happy'),
        co_classify.Example('Today, some bullying happened at my school. I feel bad for that student', 'sad'),
        co_classify.Example('I worked hard today, but it was all worth it', 'happy'),
        co_classify.Example('I worked too hard today, work was really rough', 'sad'),
        co_classify.Example('I was not able to win an award today, but my friend won something', 'happy'),
        co_classify.Example("A crime happened at my neighbor's home", 'sad'),
        co_classify.Example('I saw a cat on the street, I wish I could have brought it home', 'sad'),
        co_classify.Example("I met one of my idols today!", 'happy')
    ]
)

In [75]:
# vector encoding of happy and sad
response_labels = response.classifications[0].labels
happiness_encoding = ([response_labels['happy'].confidence, response_labels['sad'].confidence])

In [77]:
# determining the state of your day
print("speech:", speech_text)
print('rating:', day_decoding[np.argmax(happiness_encoding)])

print('/n', response_labels)

speech: My laptop missing in class today
rating: sad
/n {'happy': cohere.LabelPrediction {
	confidence: 0.060965765
}, 'sad': cohere.LabelPrediction {
	confidence: 0.9390342
}}


In [93]:
prompt = '''"The killer whale or orca (Orcinus orca) is a toothed whale
belonging to the oceanic dolphin family, of which it is the largest member"
In summary: "The killer whale or orca is the largest type of dolphin"

"It is recognizable by its black-and-white patterned body"
In summary:"Its body has a black and white pattern"

"Killer whales have a diverse diet, although individual populations often specialize in particular types of prey"
In summary:"'''
print(prompt)

n_generations = 5

prediction = co_client.generate(
    model='large',
    prompt=prompt,
    return_likelihoods = 'GENERATION',
    stop_sequences=['"'],
    max_tokens=50,
    temperature=0.7,
    num_generations=n_generations,
    k=0,
    p=0.75)


# Get list of generations
gens = []
likelihoods = []
for gen in prediction.generations:
    gens.append(gen.text)

    sum_likelihood = 0
    for t in gen.token_likelihoods:
        sum_likelihood += t.likelihood
    # Get sum of likelihoods
    likelihoods.append(sum_likelihood)

pd.options.display.max_colwidth = 200
# Create a dataframe for the generated sentences and their likelihood scores
df = pd.DataFrame({'generation':gens, 'likelihood': likelihoods})
# Drop duplicates
df = df.drop_duplicates(subset=['generation'])
# Sort by highest sum likelihood
df = df.sort_values('likelihood', ascending=False, ignore_index=True)
print('Candidate summaries for the sentence: \n"Killer whales have a diverse diet, although individual populations often specialize in particular types of prey."')
df

Candidate summaries for the sentence: 
"Killer whales have a diverse diet, although individual populations often specialize in particular types of prey."


,generation,likelihood
0,".\nIts name is derived from the Latin word orca, meaning """,-15.194356
1,". Killer whales live in all the world’s oceans, and feed almost exclusively on fish.\nThe average killer whale is 2.5 metres (8 ft) long, and weighs up to 2,000 kilograms (4,400 lb).",-46.843821
2,". Killer whales are found worldwide in all oceans. They are especially abundant in the coastal waters of the Pacific Ocean. They are also found in coastal waters of the Atlantic Ocean in Canada, G...",-58.335164
3,". It is also one of the most powerful predators in the ocean. Killer whales are highly social, with stable, monogamous pairs that raise their calves in the company of the rest of the pod. Females ...",-67.934894
4,". Killer whales are known to have a highly complex culture, with vocal dialects and specialised hunting techniques. The killer whale is among the few animal species that have been found to form lo...",-71.907593
